# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
cities_output_df = pd.read_csv("Cities_Output.csv")
cities_output_df.dropna()
cities_output_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hermanus,2,ZA,1584490056,93,-34.42,19.23,63.00,1.14
1,1,Sorong,0,ID,1584490130,67,-0.88,131.25,83.21,2.53
2,2,Rocha,100,UY,1584490131,77,-34.48,-54.33,64.78,10.54
3,3,Yellowknife,75,CA,1584490006,64,62.46,-114.35,1.99,5.82
4,4,Talnakh,75,RU,1584489899,92,69.49,88.40,10.40,26.84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_output_df[["Lat", "Lng"]]
weight = cities_output_df["Humidity"].astype(float)
#weight = cities_output_df["Humidity"].max()

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer using highest humidity as max intensity
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=cities_output_df["Humidity"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Add a "Hotel Name" column to the DataFrame
cities_output_df['Hotel Name'] = ""
cities_output_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,Hermanus,2,ZA,1584490056,93,-34.42,19.23,63.00,1.14,
1,1,Sorong,0,ID,1584490130,67,-0.88,131.25,83.21,2.53,
2,2,Rocha,100,UY,1584490131,77,-34.48,-54.33,64.78,10.54,
3,3,Yellowknife,75,CA,1584490006,64,62.46,-114.35,1.99,5.82,
4,4,Talnakh,75,RU,1584489899,92,69.49,88.40,10.40,26.84,
...,...,...,...,...,...,...,...,...,...,...,...
567,567,Xinmin,74,CN,1584490848,25,41.99,122.83,53.60,11.18,
568,568,Coari,100,BR,1584490849,96,-4.08,-63.14,76.12,1.54,
569,569,Mozelos,1,PT,1584490851,76,41.87,-8.40,60.01,2.24,
570,570,Eshowe,100,ZA,1584490852,73,-28.89,31.47,63.81,11.74,


In [7]:
# Create new DataFrame fitting weather criteria
hotel_df = cities_output_df.loc[(cities_output_df['Max Temp'] > 70) &
                                (cities_output_df['Max Temp'] < 77) &
                                (cities_output_df['Wind Speed'] < 10) &
                                (cities_output_df['Cloudiness'] == 0),
                                :].copy()
                           
hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,53,Airai,0,TL,1584490198,67,-8.93,125.41,72.90,4.52,
138,138,Oussouye,0,SN,1584490120,83,12.48,-16.55,73.40,6.93,
149,149,Batemans Bay,0,AU,1584490321,71,-35.72,150.18,73.00,3.00,
273,273,Bilma,0,NE,1584490476,9,18.69,12.92,73.40,9.51,
289,289,Rāyagada,0,IN,1584490496,72,19.17,83.42,72.57,1.63,
357,357,Coihueco,0,CL,1584490585,33,-36.62,-71.83,75.20,8.05,
401,401,Ulladulla,0,AU,1584490641,64,-35.35,150.47,72.00,8.05,
440,440,Morondava,0,MG,1584490690,90,-20.28,44.28,72.03,3.22,
459,459,Narrabri,0,AU,1584490544,56,-30.32,149.78,73.00,3.36,
538,538,Abu Dhabi,0,AE,1584490812,60,24.47,54.37,75.99,5.82,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# find hotel within 5000 m of coordinates

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (IndexError):
        print('Missing field/result... skipping.')
    
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,53,Airai,0,TL,1584490198,67,-8.93,125.41,72.90,4.52,Juvinal Martin
138,138,Oussouye,0,SN,1584490120,83,12.48,-16.55,73.40,6.93,Auberge du Routard
149,149,Batemans Bay,0,AU,1584490321,71,-35.72,150.18,73.00,3.00,Coachhouse Marina Resort - Batemans Bay
273,273,Bilma,0,NE,1584490476,9,18.69,12.92,73.40,9.51,Expeditions Ténére voyages
289,289,Rāyagada,0,IN,1584490496,72,19.17,83.42,72.57,1.63,"Hotel Swagath, Rayagada"


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map

name = hotel_df['Hotel Name'].tolist()
city = hotel_df['City'].tolist()
country = hotel_df['Country'].tolist()

#fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))